##### 解析SVG为config文件
若Figma中存在rect元素，将其全部转为Path

In [87]:
RUN_SILK = False #是否将path的id改为连接的城市名（丝绸之路）

In [88]:
import xml.etree.ElementTree as ET
from svgelements import Rect, Matrix
# 加载SVG文件
def parse_svg(svg_file):
    # 解析SVG文件
    tree = ET.parse(svg_file)
    root = tree.getroot()
    # 命名空间 (一般SVG文件会包含默认的命名空间)
    # 根据具体的SVG文件，可能需要调整命名空间前缀
    namespace = {'svg': 'http://www.w3.org/2000/svg'}
    
    # 查找所有 <g> 元素
    root_g = root.find('./svg:g', namespace)  # 使用命名空间避免解析问题
    g_elements = root_g.findall('./svg:g', namespace)  # 使用命名空间避免解析问题
    
    layers = {}
    # 遍历 <g> 元素并提取属性 （图层）
    for g in g_elements:
        # print(f"Found <g> element with attributes: {g.attrib}")
        elements = []
        # 如果需要进一步提取 <g> （图层） 内子元素及其信息
        for child in g:
            # childTag = child.tag.replace('{http://www.w3.org/2000/svg}','')
            text = child.find('.//svg:text', namespace)
            rect = child.find('.//svg:rect', namespace)
            path = child.find('.//svg:path', namespace)
            circle = child.find('.//svg:circle', namespace)
            textData = []
            if text is not None:
                text_attr = text.attrib
                # print(text_attr['transform'])
                transform = text_attr.get('transform', '')
                font_size = text_attr.get('font-size', '')
                for tspan in text:
                    tspan_attr = tspan.attrib
                    textData.append({
                        "pos": [tspan_attr['x'], tspan_attr['y']],
                        "text": tspan.text,
                        "font-size": font_size,
                        "transform": transform
                    })
            if rect is not None:
                rect_attr = rect.attrib
                rect_tranform = rect_attr.get("transform", "")
                if rect_tranform:
                    d = Rect(rect_attr['x'], rect_attr['y'], rect_attr['width'], rect_attr['height'], transform=rect_tranform).d()
                else:
                    d = Rect(rect_attr['x'], rect_attr['y'], rect_attr['width'], rect_attr['height']).d()
                del rect_attr['x']
                del rect_attr['y']
                del rect_attr['width']
                del rect_attr['height']
                rect_attr['d'] = d
                elements.append({**rect_attr, 'tag': 'path', 'text': textData})
                continue
            if path is not None:
                elements.append({**path.attrib, 'tag': 'path', 'text': textData})
                if textData:
                    textName = textData[0].get('text')
                    if path.attrib['id'].startswith('Vector') and textName is not None:
                        elements[-1]['id'] = textName
            if circle is not None:
                elements.append({**circle.attrib, 'tag': 'circle', 'text': textData})
            if g.attrib['id'] == '城市边界CHGIS':
                elements[-1]['id'] += 'CHGIS'
            if len(textData) > 0:
                elements[-1]['id'] = textData[0]['text']
        layers[g.attrib['id']] = elements
    return layers

# 示例SVG文件路径
svg_file = './silk.svg'  # 替换为你的SVG文件路径
layers = parse_svg(svg_file)


In [89]:
print(layers)

{'路径': [{'id': 'Vector 12', 'd': 'M2069 609C2035.37 613.125 2020.62 612.67 2007.5 603C1993.34 587.66 1979 581 1970 567C1924.48 573.976 1791 726 1727 718.5C1594.04 715.137 1617.09 754.05 1614 808C1600.66 812.921 1590.27 813.57 1568 812C1550.15 806.696 1539.33 800.903 1519 787C1478.74 760.885 1460.5 747.779 1448 731.5C1425 687.776 1411.62 663.497 1384 622C1375.82 616.368 1369.56 614.857 1354 616.5C1342.02 615.586 1331.91 612.485 1310 604C1302.22 602.037 1297.61 599.284 1288.5 588.5C1280.75 575.041 1275.2 566.547 1263.5 550C1244.01 549.446 1234.33 549.693 1176.5 532L1129.5 564.5C1073.88 587.333 1039.58 590.316 972.5 577L955.5 569C921.086 597.495 933.265 615.345 977 648.5C992.677 673.19 995.79 686.252 1001 709.5C1001.94 731.942 997.765 741.909 982 755', 'stroke': '#FD2B0D', 'stroke-width': '3.1', 'tag': 'path', 'text': []}, {'id': 'Vector 1', 'd': 'M2069 610C2056.43 595.131 2046.86 585.743 2025 567C1982.08 533.594 1964.78 523.772 1945.5 521.5C1935.12 517.901 1927.55 514.391 1904.5 500C1896

##### 丝绸之路 
分解path的d属性为一小段

In [90]:
import re
import math
from svg.path import parse_path, Move, Line, CubicBezier, QuadraticBezier, Arc, Close

# --- 从之前代码中保留和改进的辅助函数 ---
def format_coordinate(num):
    """
    辅助函数，用于格式化坐标值。
    将浮点数整数（如 205.0）转换为整数字符串（"205"）。
    对于其他浮点数，保留适当的精度并去除末尾多余的0和小数点。
    """
    if isinstance(num, complex):
        return f"{format_coordinate(num.real)} {format_coordinate(num.imag)}"
    if isinstance(num, float) and num.is_integer():
        return str(int(num))
    formatted_num = f"{num:.10g}" 
    if '.' in formatted_num:
        formatted_num = formatted_num.rstrip('0').rstrip('.')
    return formatted_num

def point_to_str(p):
    """将一个复数点 (complex number) 转换为 'x y' 格式的字符串。"""
    if p is None:
        return "0 0" 
    return f"{format_coordinate(p.real)} {format_coordinate(p.imag)}"

def get_command_params_str(segment_obj):
    """
    根据路径段对象，生成其命令和参数部分的字符串。
    """
    if isinstance(segment_obj, Line):
        return f"L {point_to_str(segment_obj.end)}"
    elif isinstance(segment_obj, CubicBezier):
        return f"C {point_to_str(segment_obj.control1)} {point_to_str(segment_obj.control2)} {point_to_str(segment_obj.end)}"
    elif isinstance(segment_obj, QuadraticBezier):
        return f"Q {point_to_str(segment_obj.control)} {point_to_str(segment_obj.end)}"
    elif isinstance(segment_obj, Arc):
        return (f"A {format_coordinate(segment_obj.radius.real)} {format_coordinate(segment_obj.radius.imag)} "
                f"{format_coordinate(segment_obj.rotation)} {int(segment_obj.arc)} {int(segment_obj.sweep)} "
                f"{point_to_str(segment_obj.end)}")
    elif isinstance(segment_obj, Close):
        return "Z"
    return ""

def preprocess_d_attribute(d_attr):
    """
    预处理d属性字符串，尝试修复常见语法问题，
    例如数字后紧跟命令字母（如 "0.5M" 修复为 "0.5 M"）。
    """
    # 正则表达式查找： (数字) (命令字母) -> (数字) (空格) (命令字母)
    corrected_d = re.sub(r'(\d(?:\.\d*)?)([MmZzLlHhVvCcSsQqTtAa])', r'\1 \2', d_attr)
    return corrected_d

def extract_start_end_points_from_d(d_str):
    """
    从SVG path的d属性中获取起点和终点坐标
    :param d_str: SVG path的d属性字符串
    :return: (起点坐标, 终点坐标)，例如 ((x1, y1), (x2, y2))
    """
    path_obj = parse_path(d_str)
    if not path_obj or len(path_obj) == 0:
        return None, None
    start_pt = (path_obj[0].start.real, path_obj[0].start.imag)
    end_pt = (path_obj[-1].end.real, path_obj[-1].end.imag)
    return start_pt, end_pt

# --- 用于匹配城市的函数 ---
def euclidean_distance(p1, p2):
    """计算两点 p1(x1,y1) 和 p2(x2,y2) 之间的欧几里得距离。"""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def find_closest_city(point, cities_map_data, tolerance=float('inf')):
    """
    从城市列表中找到离给定点最近的城市。
    point: (x, y) 元组
    cities_map_data: 城市数据列表，格式为 [ ((x,y), "城市名"), ... ]
    tolerance: 只有当距离小于此值时才认为匹配成功
    返回: (("城市名", (cx, cy)), 距离) 或 ((None, None), float('inf'))
    """
    if not cities_map_data:
        return ((None, None), float('inf'))

    closest_city_info = (None, None)
    min_distance = float('inf')

    for city_coord, city_name in cities_map_data:
        # 确保 city_coord 是 (x,y) 元组
        if not (isinstance(city_coord, (tuple, list)) and len(city_coord) == 2):
            print(f"警告: 城市数据 '{city_name}' 的坐标格式不正确: {city_coord}。已跳过。")
            continue
        try:
            # 确保坐标是数字
            coord_x = float(city_coord[0])
            coord_y = float(city_coord[1])
        except ValueError:
            print(f"警告: 城市数据 '{city_name}' 的坐标值无法转换为数字: {city_coord}。已跳过。")
            continue

        distance = euclidean_distance(point, (coord_x, coord_y))
        if distance < min_distance:
            min_distance = distance
            closest_city_info = (city_name, (coord_x, coord_y))
            
    if min_distance <= tolerance:
        return closest_city_info, min_distance
    else:
        return ((None, None), min_distance) # 返回None表示没有在容差范围内找到城市


if RUN_SILK:
    citys = layers['城市']
    cityData = []
    for city in citys:
        cityData.append(((float(city['cx']), float(city['cy'])), city['id']))


处理path的id为 [start_city]-[end_city] e.g., 天水-兰州

In [91]:

def process_d_attr(d_attr):
    # path_segments_with_endpoints = split_svg_path_into_segment_data(d_attr)
    # print(path_segments_with_endpoints)

    # print("路径段及其连接的城市：")
    # print("=" * 30)

    (start_point, end_point) = extract_start_end_points_from_d(d_attr)
    
    # print(f"  SVG 起点: ({format_coordinate(start_point[0])}, {format_coordinate(start_point[1])})")
    # print(f"  SVG 终点: ({format_coordinate(end_point[0])}, {format_coordinate(end_point[1])})")

    # 匹配起点城市 (可以设置一个较小的容差，比如1个像素单位，如果需要精确匹配)
    # 如果你的城市坐标就是从SVG路径命令的端点精确提取的，容差可以设为很小的值（例如0.1）
    # 如果城市坐标是手动标记的，可能需要更大的容差
    match_tolerance = float('inf') # 匹配容差：例如，距离小于1个单位才认为匹配
    
    (start_city_info, start_dist) = find_closest_city(start_point, cityData, tolerance=match_tolerance)
    (end_city_info, end_dist) = find_closest_city(end_point, cityData, tolerance=match_tolerance)

    start_city_name, start_city_coord = start_city_info
    end_city_name, end_city_coord = end_city_info
    return start_city_name, end_city_name
    # if start_city_name:
    #     print(f"  连接起点城市: {start_city_name} @ ({format_coordinate(start_city_coord[0])}, {format_coordinate(start_city_coord[1])}) (距离: {start_dist:.2f})")
    # else:
    #     print(f"  连接起点城市: 未在容差 {match_tolerance} 内匹配到城市 (最近距离: {start_dist:.2f})")
    #     # 如果需要显示最近的城市，即使它超出容差：
    #     # (fallback_start_city_info, fallback_start_dist) = find_closest_city(start_point, cityData)
    #     # if fallback_start_city_info[0]:
    #     #     print(f"    (最接近但超出容差: {fallback_start_city_info[0]} @ {fallback_start_city_info[1]}, 距离: {fallback_start_dist:.2f})")


    # if end_city_name:
    #     print(f"  连接终点城市: {end_city_name} @ ({format_coordinate(end_city_coord[0])}, {format_coordinate(end_city_coord[1])}) (距离: {end_dist:.2f})")
    # else:
    #     print(f"  连接终点城市: 未在容差 {match_tolerance} 内匹配到城市 (最近距离: {end_dist:.2f})")
    #     (fallback_end_city_info, fallback_end_dist) = find_closest_city(end_point, cityData)
    #     if fallback_end_city_info[0]:
    #         print(f"    (最接近但超出容差: {fallback_end_city_info[0]} @ {fallback_end_city_info[1]}, 距离: {fallback_end_dist:.2f})")


if RUN_SILK:
    paths = []
    for path in layers['路径']:
        start_city, end_city = process_d_attr(path['d'])
        path['id'] = start_city + '-' + end_city
        paths.append(path)

    layers['路径'] = paths
layers


{'路径': [{'id': 'Vector 12',
   'd': 'M2069 609C2035.37 613.125 2020.62 612.67 2007.5 603C1993.34 587.66 1979 581 1970 567C1924.48 573.976 1791 726 1727 718.5C1594.04 715.137 1617.09 754.05 1614 808C1600.66 812.921 1590.27 813.57 1568 812C1550.15 806.696 1539.33 800.903 1519 787C1478.74 760.885 1460.5 747.779 1448 731.5C1425 687.776 1411.62 663.497 1384 622C1375.82 616.368 1369.56 614.857 1354 616.5C1342.02 615.586 1331.91 612.485 1310 604C1302.22 602.037 1297.61 599.284 1288.5 588.5C1280.75 575.041 1275.2 566.547 1263.5 550C1244.01 549.446 1234.33 549.693 1176.5 532L1129.5 564.5C1073.88 587.333 1039.58 590.316 972.5 577L955.5 569C921.086 597.495 933.265 615.345 977 648.5C992.677 673.19 995.79 686.252 1001 709.5C1001.94 731.942 997.765 741.909 982 755',
   'stroke': '#FD2B0D',
   'stroke-width': '3.1',
   'tag': 'path',
   'text': []},
  {'id': 'Vector 1',
   'd': 'M2069 610C2056.43 595.131 2046.86 585.743 2025 567C1982.08 533.594 1964.78 523.772 1945.5 521.5C1935.12 517.901 1927.55 514

##### 保存结果

In [92]:
import json
with open("./silk.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(layers, ensure_ascii=False))